In [ ]:
import os
import subprocess
from pycwb.utils.yaml_helper import load_yaml

import os
import subprocess
import shutil
import tempfile
import logging
from pycwb.modules.logger import logger_init

logger_init()

logger = logging.getLogger(__name__)

schema = {
    "type": "object",
    "properties": {
        "target_dir": {
            "type": ["string", "null"],
            "description": "Directory to check for the module. Defaults to None."
        },
        "modules": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the module to check."
                    },
                    "repo_url": {
                        "type": ["string", "null"],
                        "description": "URL of the repository to pull from."
                    },
                    "version": {
                        "type": ["string", "null"],
                        "description": "Version of the module to check."
                    },
                    "module_path": {
                        "type": ["string", "null"],
                        "description": "Path to the module."
                    },
                },
                "required": ["name"],
            },
            "description": "List of modules to check or pull."
        }
    },
}

def load_config(external_module_config, config_schema=schema):
    """
    Load the external module configuration from a file.

    Args:
        external_module_config (str): Path to the external module configuration YAML file.

    Returns:
        dict: Loaded configuration as a dictionary.
    """
    params = load_yaml(external_module_config, config_schema)

    target_dir = params.get("target_dir")
    modules = params.get("modules", [])
    return target_dir, modules

In [ ]:
target_dir, modules = load_config('./sample_config.yaml')
print(f"Target Directory: {target_dir}")
modules

In [ ]:
def check_module_existence(module_name, target_dir=None):
    """
    Check if the specified module exists in the the pycwb installation directory.

    Args:
        module_name (str): Name of the module to check.

    Returns:
        bool: True if the module exists, False otherwise.
    """
    if target_dir is None:
        import pycwb
        target_dir = os.path.join(os.path.dirname(pycwb.__file__), 'modules')

    module_path = os.path.join(target_dir, module_name)
    return os.path.exists(module_path)

for module in modules:
    print(module['name'], check_module_existence(module['name'], target_dir))

In [ ]:
def pull_external_module(module_name, module_path, repo_url, version=None, target_dir=None):
    """
    Pull the specified external module from the repository.

    Args:
        module_name (str): Name of the module to pull.
        module_path (str): Path to the module in the repository.
        repo_url (str): URL of the repository to pull from.
        version (str, optional): Version of the module to check. Defaults to None.
        target_dir (str, optional): Directory to check for the module. Defaults to None.

    Returns:
        bool: True if the module was pulled successfully, False otherwise.
    """
    if target_dir is None:
        import pycwb
        target_dir = os.path.join(os.path.dirname(pycwb.__file__), 'modules')

    dest_path = os.path.join(target_dir, module_name)

    try:
        with tempfile.TemporaryDirectory() as tmp_dir:
            logger.info(f"Cloning {repo_url} with version {version} to {tmp_dir}")
            # subprocess.run(['git', 'clone', '--depth', '1', repo_url, tmp_dir], check=True)
            if version:
                result = subprocess.run([
                    "git", "clone", "--branch", version, "--depth", "1", repo_url, tmp_dir
                ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

                # Only print if there's a real issue
                if result.returncode != 0:
                    logger.error(f"Error cloning repository: {result.stderr}")
            else:
                subprocess.run(['git', 'clone', '--depth', '1', repo_url, tmp_dir], check=True)

            source_path = os.path.join(tmp_dir, module_path)
            if not os.path.exists(source_path):
                logger.error(f"Module path '{module_path}' does not exist in the repo.")
                return False

            if os.path.exists(dest_path):
                logger.info(f"Removing existing directory at {dest_path}")
                shutil.rmtree(dest_path)

            shutil.copytree(source_path, dest_path)
            logger.info(f"Module copied to {dest_path}")
        return True

    except subprocess.CalledProcessError as e:
        logger.error(f"Git clone failed: {e}")
    except Exception as e:
        logger.error(f"Error while copying module: {e}")

    return False


for module in modules:
    module_name = module['name']
    repo_url = module.get('repo_url')
    module_path = module.get('module_path')
    version = module.get('version')

    if repo_url and module_path:
        logger.info(f"Pulling external module {module_name} from {repo_url}")
        success = pull_external_module(module_name, module_path, repo_url, version, target_dir)
        if success:
            logger.info(f"Module {module_name} pulled successfully.")
        else:
            logger.error(f"Failed to pull module {module_name}.")
    else:
        logger.error(f"Module {module_name} does not exist and no repository URL provided.")
